# Projeto de Redes Complexas 

## Integrantes

- Akemi 
- Caio Vinicius de Oliveira, Nº USP: 11932272
- Emerson
- Juan Lucas Montanaro, Nº USP: 11912787

---------------------------------------------------------------------------

## Qual tipo de rede é mais adequado para redes sociais?

Primeiro passo, rodamos uma simulação em python que baixa 3 redes sociais :

- Adolescente : Cada nó é uma pessoa e as conexões ocorrem se ela se diz amiga de outra pessoa
- Facebook : Cada nó é um usuário e as conexões ocorrem se um usuário é amigo do outro
- Hamsterster : Cada nó é um usuário e as conexões ocorrem se um usuário é amigo do outro

Após isso, para esses 3 modelos, simulamos 3 modelos de redes (BA,ER,WS) e calculamos várias medidas características para essas redes,
sendo elas:

- Grau Médio
- Assortatividade
- Coeficiente de Clusterização
- Transitividade
- Entropia de Shannon



In [1]:
# importações
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx

from CONSTANTES import *
from simulacao import *

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score

import pandas as pd
import seaborn as sns

In [2]:
opcoes = list(SIMULE.keys())
for modelo in opcoes:
    main(modelo)


# Iniciando simulações do Modelo Erdos-renyi #

Rede Social -- twitter-pequeno
N: 23370, <k>: 2.8097
Tempo com a rede: 0.12 segs.


Começando agora a compilar o arquivo de rede real [twitter-pequeno] com os resultados.
Començando as simulações para a rede twitter-pequeno 

Twitter-pequeno 	Simulação #5 	OK
Twitter-pequeno 	Medidas #5 	OK
Twitter-pequeno 	Simulação #4 	OK
Twitter-pequeno 	Medidas #4 	OK
Twitter-pequeno 	Simulação #2 	OK
Twitter-pequeno 	Medidas #2 	OK
Twitter-pequeno 	Simulação #1 	OK
Twitter-pequeno 	Medidas #1 	OK
Twitter-pequeno 	Simulação #3 	OK
Twitter-pequeno 	Medidas #3 	OK
Twitter-pequeno 	Simulação #10 	OK
Twitter-pequeno 	Medidas #10 	OK
Twitter-pequeno 	Simulação #8 	OK
Twitter-pequeno 	Medidas #8 	OK
Twitter-pequeno 	Simulação #6 	OK
Twitter-pequeno 	Medidas #6 	OK
Twitter-pequeno 	Simulação #9 	OK
Twitter-pequeno 	Medidas #9 	OK
Twitter-pequeno 	Simulação #7 	OK
Twitter-pequeno 	Medidas #7 	OK

Fim das simulações para a rede twitter-pequeno
Tempo de simula

### Leitura e Tratamento dos dados

In [3]:
df_ba = pd.read_csv('./Modelo_barabasi-albert.csv')
df_ba['modelo_rede'] = 'BA'

df_er = pd.read_csv('./Modelo_barabasi-albert.csv')
df_er['modelo_rede'] = 'ER'

df_ws = pd.read_csv('./Modelo_barabasi-albert.csv')
df_ws['modelo_rede'] = 'WS'

### Rodando um PCA para as Variáveis de Interesse

In [4]:
df = pd.concat([df_ba,df_er,df_ws], axis=0, ignore_index=True)
df


X = df.drop(columns=['grau_medio','Rede Social','Iteracao','modelo_rede'])
y = df[['modelo_rede']]

# pca
pca = PCA(n_components=2)
pca.fit(X)

PCA(n_components=2)

In [5]:
print("Auto-valores:")
print(pca.explained_variance_)
print()

print("Auto-vetores:")
print(pca.components_)
print()

print("Variância explicada:")
print(pca.explained_variance_ratio_)
print()

Auto-valores:
[9.95437027e-01 4.01388527e-04]

Auto-vetores:
[[ 0.01490314  0.01099532  0.00860653  0.99979144]
 [-0.99983935  0.00996604  0.00188663  0.01477802]]

Variância explicada:
[9.99595929e-01 4.03065515e-04]



In [6]:
# Tranformando X
X1 = pca.transform(X)
# Visualizando
new_df = pd.DataFrame(X1, columns = ['pc1','pc2'])
new_df['modelo_rede'] = df['modelo_rede']

In [8]:
fig = px.scatter(new_df,x = 'pc1', y = 'pc2', color = 'modelo_rede')
fig.show()

## Treinando um Classificador para a Rede em Questão

In [9]:
X = new_df[['pc1','pc2']]
y = new_df[['modelo_rede']]

model = RandomForestClassifier()
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

c:\Users\caiov\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\caiov\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\caiov\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\caiov\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1

In [10]:
# Results
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#podemos melhorar

Accuracy: 0.33 (+/- 0.00)


In [11]:
model.fit(X,y)

C:\Users\caiov\AppData\Local\Temp\ipykernel_116196\1298606489.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier()

### Rodar as redes reais - transformar o PCA e fazer a predição para descobrir qual modelo ela mais se adequa

In [12]:
fac = pd.read_csv('./rede_real_facebook.csv')

ham = pd.read_csv('./rede_real_hamster.csv')

tt = pd.read_csv('./rede_real_twitter-pequeno.csv')

df_real = pd.concat([fac,ham,tt], axis=0, ignore_index=True)
df_real = df_real.drop(columns='grau_medio')


In [13]:
x_real = pd.DataFrame(pca.transform(df_real), columns=['pc1','pc2'])
x_real

,pc1,pc2
0,-2.192877,0.576732
1,2.555829,-0.063327
2,-0.991300,0.404698


In [14]:
df_real['predict'] = model.predict(x_real)

In [15]:
df_real

,assortatividade,coef_clusterizacao,transitividade,entropy,predict
0,-0.668214,0.027247,0.000359,0.254405,BA
1,0.047404,0.537626,0.231432,4.985832,BA
2,-0.477982,0.069224,0.021453,1.452755,BA


In [16]:
# teste:
for rede in redes_sociais.keys():

    ######################################
    # lidando com a rede social
    ######################################
    print('Rede Social --', rede)

    G = nx.read_edgelist("redes/" + redes_sociais[rede], comments='%', nodetype=int)
    grau_medio = np.array(G.degree)[:, 1].mean()
    N = len(G)
    print('<k>:', grau_medio, '\tN:', N)

Rede Social -- twitter-pequeno


<k>: 2.809670517757809 	N: 23370
Rede Social -- facebook
<k>: 2.064404432132964 	N: 2888
Rede Social -- hamster
<k>: 13.710634789777412 	N: 2426


In [17]:
# teste de simulação:
aux = nx.erdos_renyi_graph(N, grau_medio/(N-1), seed=42)

# plt.figure(figsize=(12,8))
# pos = nx.spring_layout(aux, iterations=30, k=0.5)
# nx.draw(aux, node_size=20, with_labels=False)

MEDIDAS['grau_medio'](aux)

13.6760098928277